This file is used to build all of the json data that gets loaded by `train.ipynb`

In [2]:
# Create palette file from palette.png
from util/convert_image_to_palette import convert 

convert("data/palette.png", "data/palette.json")

In [ ]:
# Create training data file